# Assignment 6

In this assignment, you'll analyze a collection of data sets from the [San Francisco Open Data Portal](http://data.sfgov.org/) and [Zillow](https://www.zillow.com/). The data sets have been stored in the SQLite database `sf_data.sqlite`, which you can [download here](http://anson.ucdavis.edu/~nulle/sf_data.sqlite). The database contains the following tables:

Table                   | Description
----------------------- | -----------
`crime`                 | Crime reports dating back to 2010.
`mobile_food_locations` | List of all locations where mobile food vendors sell.
`mobile_food_permits`   | List of all mobile food vendor permits. More details [here](https://data.sfgov.org/api/views/rqzj-sfat/files/8g2f5RV4PEk0_b24iJEtgEet9gnh_eA27GlqoOjjK4k?download=true&filename=DPW_DataDictionary_Mobile-Food-Facility-Permit.pdf).
`mobile_food_schedule`  | Schedules for mobile food vendors.
`noise`                 | Noise complaints dating back to August 2015.
`parking`               | List of all parking lots.
`parks`                 | List of all parks.
`schools`               | List of all schools.
`zillow`                | Zillow rent and housing statistics dating back to 1996. More details [here](https://www.zillow.com/research/data/).

The `mobile_food_` tables are explicitly connected through the `locationid` and `permit` columns. The other tables are not connected, but you may be able to connect them using dates, latitude/longitude, or postal codes.

Shapefiles for US postal codes are available [here](https://www.census.gov/geo/maps-data/data/cbf/cbf_zcta.html). These may be useful for converting latitude/longitude to postal codes.

Shapefiles for San Francisco Neighborhoods are available [here](https://data.sfgov.org/Geographic-Locations-and-Boundaries/SF-Find-Neighborhoods/pty2-tcw4).

__Exercise 1.1.__ Which mobile food vendor(s) sells at the most locations?

In [18]:
import pandas as pd
import numpy as np
from sqlalchemy import *
from IPython.display import IFrame
import folium
from folium import plugins
import pygeoj
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [19]:
engine = create_engine('sqlite:///sf_data.sqlite')
db = engine.connect()

In [20]:
pd.read_sql('SELECT * FROM mobile_food_locations '
            'LIMIT 1', db)

,locationid,LocationDescription,Address,Latitude,Longitude
0,762182,TOWNSEND ST: 05TH ST to 06TH ST (400 - 499),444 TOWNSEND ST,37.774871,-122.398532


In [21]:
pd.read_sql('SELECT * FROM mobile_food_permits '
            'LIMIT 1', db)

,permit,Status,Applicant,FacilityType,FoodItems,PriorPermit,Approved,Expiration
0,16MFF-0027,APPROVED,F & C Catering,Truck,Cold Truck: Hot/Cold Sandwiches: Water: Soda: ...,1,2016-03-09 12:00:00,2017-03-15 12:00:00


In [22]:
pd.read_sql('SELECT * FROM mobile_food_schedule '
            'LIMIT 1', db)

,locationid,permit,DayOfWeek,EndHour,StartHour
0,305727,11MFF-0040,Mo,15,10


In [23]:
pd.read_sql('SELECT Applicant, count(DISTINCT locationid) AS local_number '
         'FROM mobile_food_schedule AS s '
         'INNER JOIN mobile_food_permits AS p ON s.permit = p.permit '
         'GROUP BY Applicant '
         'ORDER BY local_number DESC LIMIT 1', db)

,Applicant,local_number
0,May Catering,58


May Catering sells at most locations.

__Exercise 1.2.__ Ask and use the database to analyze 5 questions about San Francisco. For each question, write at least 150 words and support your answer with plots. Make a map for at least 2 of the 5 questions.

You should try to come up with some questions on your own, but these are examples of reasonable questions:

* Which parts of the city are the most and least expensive?
* Which parts of the city are the most dangerous (and at what times)?
* Are noise complaints and mobile food vendors related?
* What are the best times and places to find food trucks?
* Is there a relationship between housing prices and any of the other tables?

Please make sure to clearly state each of your questions in your submission.

* <h4>What is the distribution of housing pricing?</h4>

In [24]:
# Cheapest
pd.read_sql('SELECT RegionName, avg(MedianSoldPricePerSqft_AllHomes) AS avg_price FROM zillow '
            'GROUP BY RegionName HAVING avg_price IS NOT NULL '
            'ORDER BY avg_price LIMIT 1', db)

,RegionName,avg_price
0,94124,297.008046


In [25]:
# Most expensive
pd.read_sql('SELECT RegionName, avg(MedianSoldPricePerSqft_AllHomes) AS avg_price FROM zillow '
            'GROUP BY RegionName HAVING avg_price IS NOT NULL '
            'ORDER BY avg_price DESC LIMIT 1', db)

,RegionName,avg_price
0,94104,1336.128581


In [26]:
price = pd.read_sql('SELECT RegionName, avg(MedianSoldPricePerSqft_AllHomes) AS avg_price FROM zillow '
            'GROUP BY RegionName HAVING avg_price IS NOT NULL '
            'ORDER BY avg_price DESC', db)
price.RegionName = price.RegionName.astype(str)

In [35]:
zip_geo = r'San Francisco ZIP Codes.geojson'
SF_COORDINATES = (37.76, -122.45)
map_price = folium.Map(location=SF_COORDINATES, zoom_start=12,
                   tiles='Stamen Terrain')
map_price.choropleth(geo_path = zip_geo,
                     data = price,
                     columns = ['RegionName', 'avg_price'],
                     key_on = 'feature.properties.zip',
                     fill_color = 'YlOrRd',
                     fill_opacity = 0.7,
                     line_opacity = 0.2,
                     legend_name = 'Lvl of price per district')          
map_price.save(outfile= "map_price.html")

In [36]:
IFrame('map_price.html', width=800, height=500)

The SF house pricing map shows that expensive houses and apartments tend to appear in the northeast part as well as middle part of SF. South SF are much cheaper than northeast SF. I did look at google map and found that the most expensive region is 94104 which is located between Financial District and China town. There is no doubt that my result is correct since Financial District is the downtown of SF while China town is another popular place in SF because of huge amount of Chinese and Chinese immigrants. 

* <h4>Which parts of the city are the most dangerous and at what times crime occurs the most?</h4>

In [30]:
crime_loc = pd.read_sql('SELECT PdDistrict, COUNT(*) as count FROM crime '
            'GROUP BY PdDistrict '
            'HAVING pdDistrict IS NOT NULL '
            'ORDER BY count DESC', db)
crime_loc.head()

,PdDistrict,count
0,SOUTHERN,196178
1,MISSION,136216
2,NORTHERN,124365
3,CENTRAL,109242
4,BAYVIEW,101655


In [31]:
pd.read_sql('SELECT DayOfWeek, COUNT(*) as count FROM crime '
            'GROUP BY DayOfWeek '
            'ORDER BY count DESC LIMIT 5', db)

,DayOfWeek,count
0,Friday,159182
1,Saturday,151941
2,Wednesday,150197
3,Thursday,146130
4,Tuesday,144353


In [37]:
district_geo = r'sfpddistricts.geojson'
SF_COORDINATES = (37.76, -122.45)
map_crime = folium.Map(location=SF_COORDINATES, zoom_start=12,
                   tiles='Stamen Terrain')
map_crime.choropleth(geo_path = district_geo,
                     data = crime_loc,
                     columns = ['PdDistrict', 'count'],
                     key_on = 'feature.properties.DISTRICT',
                     fill_color = 'YlOrRd',
                     fill_opacity = 0.7,
                     line_opacity = 0.2,
                     legend_name = 'Number of incidents per district')          

map_crime.save(outfile= "map_crime.html")

In [39]:
IFrame('map_crime.html', width=800, height=500)

SOUTHERN is the most dangerous place in SF and the number of crimes occur there is higher than the second place by 50%. Friday is the most dangerou day of week in SF. I guess people tend to hang out on Friday's night after a week of exhausting works and crimes like robbery and burglar are more likely to happen. Look at the map, we can see that downtown and China Town are the most dangerous places. However, there are tons of incidents happened on treasure island. I searched the treasure island crime map and found many thefts happen there

* <h4>Are crime and noise complains related?</h4>

In [41]:
vendor = pd.read_sql('SELECT Latitude AS Lat, Longitude AS Lon, '
                     'count(*) as count FROM mobile_food_locations AS l '
                     'INNER JOIN mobile_food_schedule AS s '
                     'ON s.locationid = l.locationid '
                     'GROUP BY Latitude, Longitude '
                     'ORDER BY count DESC', db)
vendor = vendor[vendor['Lat'] != 0]

In [43]:
# Color the grouped spots in order to reduce the dimension of data
def color(count):
    '''manually assign color by frequency'''
    mini = min(vendor['count'])
    step = (max(vendor['count'])-min(vendor['count']))/4
    if count in xrange(mini, mini+step):
        col = 'greem'
    elif count in xrange(mini+step, mini+step*2):
        col = 'yellow'
    elif count in xrange(mini+step*2, mini+step*3):
        col = 'orange'
    else:
        col = 'red'

    return col

In [44]:
noise = pd.read_sql('SELECT Lat,Lon,count(*) as count FROM noise '
            'WHERE Lat IS NOT NULL '
            'GROUP BY Lat, Lon '
            'ORDER BY count DESC', db)
# delete the observations with extremely low frequency which avoid the map to be overwhalmed.
noise = noise[noise['count'] != 1]

In [45]:
district_geo = r'sfpddistricts.geojson'
SF_COORDINATES = (37.76, -122.45)
map_crime_noise = folium.Map(location=SF_COORDINATES, zoom_start=12,
                   tiles='Stamen Terrain')
map_crime_noise.choropleth(geo_path = district_geo,
                     data = crime_loc,
                     columns = ['PdDistrict', 'count'],
                     key_on = 'feature.properties.DISTRICT',
                     fill_color = 'YlOrRd',
                     fill_opacity = 0.7,
                     line_opacity = 0.2,
                     legend_name = 'Number of incidents per district')          

noise.apply(lambda row:folium.CircleMarker(location=[row["Lat"],  \
                                      row["Lon"]], popup = str(row['count']), radius = 2.5, \
                                           fill_color = color(row['count']), color = color(row['count']) \
                                           ,).add_to(map_crime_noise),axis=1)

#vendor.apply(lambda row:folium.CircleMarker(location=[row["Lat"],  \
#                                      row["Lon"]], popup = str(row['count']), radius = 2.5, \
#                                           fill_color = color(row['count']), color = color(row['count']) \
#                                           ,).add_to(map_crime),axis=1)

map_crime_noise.save(outfile= "map_crime_noise.html")

In [46]:
IFrame('map_crime_noise.html', width=800, height=500)

It is obvious that spots of noise complaints are clustered at orange and red districts. Therefore, there is no doubt we can draw the conclusion that noise complaints occur where crime happens a lot. However, are they related directely or indirectely? I tend to say that they are indirectely connected. More noise complaints means those districts are more 
bustling and crowed like downtown. Those crowed place seems to have more crime report like [los angelas](http://maps.latimes.com/neighborhoods/neighborhood/downtown/crime/). Therefore, I make this assumption: Both crime and noise complaint are directely related to the blustling level of a district.

* <h4>Which place has the most schools?</h4>

In [47]:
school = pd.read_sql('SELECT * FROM schools', db)

In [48]:
# extract the zip code from Address
school_zip = school['Address'].apply(lambda row: re.findall(r'[9][0-9]{4}', row))
school['zip'] = pd.Series([i[0] for i in school_zip])
zip_count = school[['zip','Name']].groupby('zip').count().reset_index()
zip_count.head()

,zip,Name
0,94102,23
1,94103,14
2,94105,6
3,94107,20
4,94108,12


In [49]:
zip_geo = r'San Francisco ZIP Codes.geojson'
SF_COORDINATES = (37.76, -122.45)
map_school = folium.Map(location=SF_COORDINATES, zoom_start=12,
                   tiles='Stamen Terrain')
map_school.choropleth(geo_path = zip_geo,
                     data = zip_count,
                     columns = ['zip', 'Name'],
                     key_on = 'feature.properties.zip',
                     fill_color = 'YlOrRd',
                     fill_opacity = 0.7,
                     line_opacity = 0.2,
                     legend_name = 'Number of schools per district')          

map_school.save(outfile= "map_school.html")

In [50]:
IFrame('map_school.html', width=800, height=500)

This map is quite interesting. Schools spread out in SF. Except middle of SF, all other districts with high housing price and high crime rate do not have many school there. This is quite reasonable because schools can not be set where a large quantity of crimes occur. As we all know the SF has super bad traffic condition in US and it definitely takes too much time for student to go to school if schools are set at bustling districts. Morover, South part of SF may be a pretty good place to live on account of low price, low crime rate and large amount of schools there.

<h4>Is house price related to parking lot distribution?</h4>

In [51]:
parking=pd.read_sql('SELECT Lat, Lon, count(*) as count FROM parking '
            'WHERE Lat != 0 '
            'GROUP BY Lat, Lon '
            'ORDER BY count Desc', db)

In [52]:
zip_geo = r'San Francisco ZIP Codes.geojson'
SF_COORDINATES = (37.76, -122.45)
map_parking = folium.Map(location=SF_COORDINATES, zoom_start=12,
                   tiles='Stamen Terrain')
map_parking.choropleth(geo_path = zip_geo,
                     data = price,
                     columns = ['RegionName', 'avg_price'],
                     key_on = 'feature.properties.zip',
                     fill_color = 'YlOrRd',
                     fill_opacity = 0.7,
                     line_opacity = 0.2,
                     legend_name = 'Lvl of price per district')          

parking.apply(lambda row:folium.CircleMarker(location=[row["Lat"],  \
                                      row["Lon"]], radius = 1).add_to(map_parking),axis=1)
map_parking.save(outfile= "map_parking.html")

In [53]:
IFrame('map_parking.html', width=800, height=500)

All in all, it is hard to tell whether there is a relationship between those two things. Although we can see that parking lot are clustered at downtown which is quite expensive compared to other district. This is quite true since CBD needs more parking lot and garage building. However, in other orange and yellow distric as shown in the map, there is no obvious pattern for the distribution of parking. Therefore, I conclude that housing price and parking do not have direct relation.

In [54]:
db.close()